# Authoring Leaf Systems
For instructions on how to run these tutorial notebooks, please see the [index](./index.ipynb).


In [1]:
import numpy as np
from pydrake.common.containers import namedview
from pydrake.common.value import Value
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import BasicVector, LeafSystem
from pydrake.trajectories import PiecewisePolynomial

## Overview

The [Modeling Dynamical Systems](./dynamical_systems.ipynb) tutorial gave a very basic introduction to Drake's Systems framework, including writing a basic [LeafSystem](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html). In this notebook we'll provide a more advanced/complete overview of authoring those systems.

## Input and Output Ports

Leaf systems can have any number of input and output ports.  There are two basic types of ports: **vector-valued** ports and **abstract-valued** ports. 

### Vector-valued Ports
Let's start with vector-valued ports, which are the easiest to work with.  The following system declares two vector input ports, and two vector output ports. It forms the sum and difference of the two 2-element input vectors and places the results on the two output ports.

<table align=center cellpadding=0 cellspacing=0><tr align=center style="border:none;"><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">a &rarr;</td></tr><tr style="border:none;"><td align=right style="padding:5px 0px 5px 0px; border:none;">b &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle;" bgcolor=#F0F0F0>MyAdder</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; sum</td></tr><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; difference</td></tr></table></td></tr></table>

In [2]:
class MyAdder(LeafSystem):
    def __init__(self):
        super().__init__()  # Don't forget to initialize the base class.
        self._a_port = self.DeclareVectorInputPort(name="a", size=2)
        self._b_port = self.DeclareVectorInputPort(name="b", size=2)
        self.DeclareVectorOutputPort(name="sum", size=2, calc=self.CalcSum)
        self.DeclareVectorOutputPort(name="difference",
                                     size=2,
                                     calc=self.CalcDifference)

    def CalcSum(self, context, output):
        # Evaluate the input ports to obtain the 2x1 vectors.
        a = self._a_port.Eval(context)
        b = self._b_port.Eval(context)

        # Write the sum into the output vector.
        output.SetFromVector(a + b)

    def CalcDifference(self, context, output):
        # Evaluate the input ports to obtain the 2x1 vectors.
        a = self._a_port.Eval(context)
        b = self._b_port.Eval(context)

        # Write the difference into output vector.
        output.SetFromVector(a - b)

# Construct an instance of this system and a context.
system = MyAdder()
context = system.CreateDefaultContext()

# Fix the input ports to some constant values.
system.GetInputPort("a").FixValue(context, [3, 4])
system.GetInputPort("b").FixValue(context, [1, 2])

# Evaluate the output ports.
print(f"sum: {system.GetOutputPort('sum').Eval(context)}")
print(f"difference: {system.GetOutputPort('difference').Eval(context)}")

sum: [4. 6.]
difference: [2. 2.]


There are a few things to notice here:
- You can declare as many inputs or outputs as you like (and they need not be the same size).
- Any of the inputs can be evaluated via their `Eval()` method from any of the system methods that accept a `Context`.
- For each output port, you define a different callback function that implements the output.
- Keeping track of the order of the ports can be a bit brittle. So we can improve the code, e.g., by using their index or referencing them by name.

It is also helpful to understand that:
- You can also make inputs optional, by checking [InputPort::HasValue()](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_input_port.html#a5536b94a4642fa4cf47164437dc66ae8) in the system method implementations.
- Input and output ports do not have any timing semantics on their own; output ports are evaluated whenever their output is requested, and [use caching](https://drake.mit.edu/doxygen_cxx/group__cache__design__notes.html) by default to avoid repeated computation.
- The `DeclareVectorOutputPort()` and `DeclareAbstractOutputPort()` ports take an optional `prerequisites_of_calc` argument which is used by the caching system to determine when the output port needs to be re-evaluated. Declaring narrow prerequisites can speed up your code. Surprisingly, explicitly narrowing the prerequisites can also dramatically speed up the construction of some `Diagram`s, because when these are not specified the `DiagramBuilder` attempts to check for algebraic loops by converting your systems into symbolic form.


### Abstract-valued Ports
Not all inputs and outputs are best represented as a vector. Drake's systems framework also supports passing structured types across ports, which is implemented in C++ using a technique called "type erasure". From the perspective of the systems framework, classes work with an "abstract" data type, and only the system implementation itself needs to know how to work with the structured type. In practice, if you can overlook a small amount of boilerplate, the usage is straightforward. Let's give an example of a `LeafSystem` that uses a `RigidTransform` in its input and output ports:

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">in &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>RotateAboutZ</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; out</td></tr></table></td></tr></table>

In [3]:
class RotateAboutZ(LeafSystem):
    def __init__(self):
        super().__init__()  # Don't forget to initialize the base class.
        self.DeclareAbstractInputPort(name="in",
                                      model_value=Value(RigidTransform()))
        self.DeclareAbstractOutputPort(
            name="out",
            alloc=lambda: Value(RigidTransform()),
            calc=self.CalcOutput)

    def CalcOutput(self, context, output):
        # Evaluate the input port to obtain the RigidTransform.
        X_1 = system.get_input_port().Eval(context)

        X_2 = RigidTransform(RotationMatrix.MakeZRotation(np.pi / 2)) @ X_1

        # Set the output RigidTransform.
        output.set_value(X_2)

# Construct an instance of this system and a context.
system = RotateAboutZ()
context = system.CreateDefaultContext()

# Fix the input port to a constant value.
system.get_input_port().FixValue(context, RigidTransform())

# Evaluate the output port.
print(f"output: {system.get_output_port(0).Eval(context)}")

output: RigidTransform(
  R=RotationMatrix([
    [6.123233995736766e-17, -1.0, 0.0],
    [1.0, 6.123233995736766e-17, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)


## State Variables

Systems store their state in the `Context`, and work with state in a very similar way to input and output ports. Again, we can have vector-valued state or abstract-valued state. Abstract state can only be updated in a discrete-time fashion, but vector-valued state can be declared to be either **discrete** or **continuous**.  

A discrete state $x_d$ will evolve based on update events, the most common of which would be a simple periodic event to define a difference equation.  We saw an example of this in the [introductory notebook](./dynamical_systems.ipynb).

In [4]:
class SimpleDiscreteTimeSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        state_index = self.DeclareDiscreteState(1)  # One state variable.
        self.DeclareStateOutputPort("y", state_index)  # One output: y=x.
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=1.0,  # One second time step.
            offset_sec=0.0,  # The first event is at time zero.
            update=self.Update) # Call the Update method defined below.

    # x[n+1] = x^3[n].
    def Update(self, context, discrete_state):
        x = context.get_discrete_state_vector().GetAtIndex(0)
        x_next = x**3
        discrete_state.get_mutable_vector().SetAtIndex(0, x_next)

# Instantiate the System.
system = SimpleDiscreteTimeSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# Set the initial conditions: x[0] = [0.9].
context.get_mutable_discrete_state_vector().SetFromVector([0.9])

# Run the simulation.
simulator.AdvanceTo(4.0)
print(context.get_discrete_state_vector())

[0.00019662705047555356]


You can find details on the implementation and order of these events [here](https://drake.mit.edu/doxygen_cxx/group__discrete__systems.html). It's also possible to define more complicated [events](https://drake.mit.edu/doxygen_cxx/group__events__description.html) for updating the discrete variables. 

Note that the states of a system are not immediately accessible to other systems in a `Diagram`. For a system to share its state, it must do that using an output port. We provide the `DeclareStateOutputPort()` method to make this common case easy.

The evolution of continuous vector-valued state variables is governed by a differential equation. While it is possible and convenient to declare many different groups of discrete state variables (they may even be updated at different rates by different events), we only define zero or one continuous state vector for a system, and define its dynamics by overloading the `LeafSystem::DoCalcTimeDerivatives()` method. Here is the simple continuous-time example in the [introductory notebook](./dynamical_systems.ipynb) again: 

In [5]:
# Define the system.
class SimpleContinuousTimeSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        state_index = self.DeclareContinuousState(1)  # One state variable.
        self.DeclareStateOutputPort("y", state_index)  # One output: y=x.

    # xdot(t) = -x(t) + x^3(t).
    def DoCalcTimeDerivatives(self, context, derivatives):
        x = context.get_continuous_state_vector().GetAtIndex(0)
        xdot = -x + x**3
        derivatives.get_mutable_vector().SetAtIndex(0, xdot)

# Instantiate the System.
system = SimpleContinuousTimeSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# Set the initial conditions: x(0) = [0.9]
context.SetContinuousState([0.9])

# Run the simulation.
simulator.AdvanceTo(4.0)
print(context.get_continuous_state_vector())

[0.03778876136193669]


For abstract-valued state, we have a similar workflow. We declare the abstract state in the constructor, and define update events to update that state.  Discrete update events are restricted to updating *only* the discrete states, so we use "unrestricted" events to update the abstract state. Here is an example which stores a simple `PiecewisePolynomial` trajectory as abstract state:

In [6]:
class AbstractStateSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        self._traj_index = self.DeclareAbstractState(
            Value(PiecewisePolynomial()))
        self.DeclarePeriodicUnrestrictedUpdateEvent(period_sec=1.0,
                                                    offset_sec=0.0,
                                                    update=self.Update)

    def Update(self, context, state):
        t = context.get_time()
        traj = PiecewisePolynomial.FirstOrderHold(
            [t, t + 1],
            np.array([[-np.pi / 2.0 + 1., -np.pi / 2.0 - 1.], [-2., 2.]]))
        # Update the state
        state.get_mutable_abstract_state(int(self._traj_index)).set_value(traj)



system = AbstractStateSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# Set an initial condition for the abstract state.
context.SetAbstractState(0, PiecewisePolynomial())

# Run the simulation.
simulator.AdvanceTo(4.0)
traj = context.get_abstract_state(0).get_value()
print(f"breaks: {traj.get_segment_times()}")
print(f"traj({context.get_time()}) = {traj.value(context.get_time())}")

breaks: [3.0, 4.0]
traj(4.0) = [[-2.57079633]
 [ 2.        ]]


## Parameters

Parameters are like state, except that they are constant through the lifetime of a simulation. Parameters in the systems framework are declared and accessed almost identically to state, but they are never updated. Once again, we have vector-valued (declared with `DeclareNumericParameter`) and abstract-valued (via `DeclareAbstractParameter`) parameters.

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>SystemWithParameters</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; numeric</td></tr><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; abstract</td></tr></table></td></tr></table>

In [7]:
class SystemWithParameters(LeafSystem):
    def __init__(self):
        super().__init__()  # Don't forget to initialize the base class.

        self.DeclareNumericParameter(BasicVector([1.2, 3.4]))
        self.DeclareAbstractParameter(
            Value(RigidTransform(RotationMatrix.MakeXRotation(np.pi / 6))))

        # Declare output ports to demonstrate how to access the parameters in
        # system methods.
        self.DeclareVectorOutputPort(name="numeric",
                                     size=2,
                                     calc=self.OutputNumeric)
        self.DeclareAbstractOutputPort(
            name="abstract",
            alloc=lambda: Value(RigidTransform()),
            calc=self.OutputAbstract)

    def OutputNumeric(self, context, output):
        output.SetFromVector(context.get_numeric_parameter(0).get_value())

    def OutputAbstract(self, context, output):
        output.set_value(context.get_abstract_parameter(0).get_value())

# Construct an instance of this system and a context.
system = SystemWithParameters()
context = system.CreateDefaultContext()

# Evaluate the output ports.
print(f"numeric: {system.get_output_port(0).Eval(context)}")
print(f"abstract: {system.get_output_port(1).Eval(context)}")

numeric: [1.2 3.4]
abstract: RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.8660254037844387, -0.49999999999999994],
    [0.0, 0.49999999999999994, 0.8660254037844387],
  ]),
  p=[0.0, 0.0, 0.0],
)


## Systems can "publish"

In addition to methods that update the state, and methods that evaluate an output port, another supported method in a `LeafSystem` is a callback to "publish". A "publish" method cannot modify any state: they are useful for broadcasting data outside of the systems framework (e.g. via a message-passing protocol like ROS), for terminating a simulation, for detecting errors, and for forcing boundaries between integration steps. They are declared in much the same way as other system callbacks.

In [8]:
class MyPublishingSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        # Calling `ForcePublish()` will trigger the callback.
        self.DeclareForcedPublishEvent(self.Publish)

        # Publish once every second.
        self.DeclarePeriodicPublishEvent(period_sec=1,
                                         offset_sec=0,
                                         publish=self.Publish)
        
    def Publish(self, context):
        print(f"Publish() called at time={context.get_time()}")

system = MyPublishingSystem()
simulator = Simulator(system)
simulator.AdvanceTo(5.3)

# We can also "force" a publish at a arbitrary time.
print("\ncalling ForcedPublish:")
system.ForcedPublish(simulator.get_context())


Publish() called at time=0.0
Publish() called at time=1.0
Publish() called at time=2.0
Publish() called at time=3.0
Publish() called at time=4.0
Publish() called at time=5.0

calling ForcedPublish:
Publish() called at time=5.3


## Naming Vector Values

Abstract-valued ports, state, and/or parameters can be used to work with structured data. But it can also be convenient to use string names to reference the individual elements of vector-valued data. In Python, we recommend using a [`namedview`](https://drake.mit.edu/pydrake/pydrake.common.containers.html?highlight=namedview#pydrake.common.containers.namedview) workflow, which was inspired by Python's [`namedtuple`](https://docs.python.org/3.6/library/collections.html?highlight=namedtuple#collections.namedtuple) and is demonstrated in the example below. The Drake developers plan to [provide similar functionality in C++](https://github.com/RobotLocomotion/drake/issues/12566).

Note that the memory layout for a continuous state vector is slightly different than for discrete state, parameters, and input/output port data, so the syntax for dealing with it requires a slightly distinct `CopyToVector()` and `SetFromVector()`.  See [issue #9171](https://github.com/RobotLocomotion/drake/issues/9171).

Note that usages of `my_view[:]` are meant as a shorthand to get a NumPy array view of the `namedview` itself.
For more details, see the example and warning in the documentation for [`namedview`](https://drake.mit.edu/pydrake/pydrake.common.containers.html?highlight=namedview#pydrake.common.containers.namedview).

In [9]:
# Define the system.
class NamedViewDemo(LeafSystem):
    MyDiscreteState = namedview("MyDiscreteState", ["a", "b"])
    MyContinuousState = namedview("MyContinuousState", ["x", "z", "theta"])
    MyOutput = namedview("MyOutput", ["x","a"])

    def __init__(self):
        super().__init__()

        self.DeclareDiscreteState(2)
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=1.0,
            offset_sec=0.0,
            update=self.DiscreteUpdate)
        self.DeclareContinuousState(3)
        self.DeclareVectorOutputPort(name="out", size=2, calc=self.CalcOutput)

    def DiscreteUpdate(self, context, discrete_values):
        discrete_state = self.MyDiscreteState(
            context.get_discrete_state_vector().value())
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        next_state = self.MyDiscreteState(discrete_values.get_mutable_value())
        # Now we can compute the next state by referencing each element by name.
        next_state.a = discrete_state.a + 1
        next_state.b = discrete_state.b + continuous_state.x

    def DoCalcTimeDerivatives(self, context, derivatives):
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        dstate_dt = self.MyContinuousState(continuous_state[:])
        dstate_dt.x = -continuous_state.x
        dstate_dt.z = -continuous_state.z
        dstate_dt.theta = -np.arctan2(continuous_state.z, continuous_state.x)
        derivatives.SetFromVector(dstate_dt[:])

    def CalcOutput(self, context, output):
        discrete_state = self.MyDiscreteState(
            context.get_discrete_state_vector().value())
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        out = self.MyOutput(output.get_mutable_value())
        out.x = continuous_state.x
        out.a = discrete_state.a

# Instantiate the System.
system = NamedViewDemo()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# Set the initial conditions.
initial_discrete_state = NamedViewDemo.MyDiscreteState([3, 4])
context.SetDiscreteState(initial_discrete_state[:])
initial_continuous_state = NamedViewDemo.MyContinuousState.Zero()
initial_continuous_state.x = 0.5
initial_continuous_state.z = 0.92
initial_continuous_state.theta = 0.23
context.SetContinuousState(initial_continuous_state[:])

# Run the simulation.
simulator.AdvanceTo(4.0)
print(
    NamedViewDemo.MyDiscreteState(context.get_discrete_state_vector().value()))
print(
    NamedViewDemo.MyContinuousState(
        context.get_continuous_state_vector().CopyToVector()))
print(NamedViewDemo.MyOutput(system.get_output_port().Eval(context)))


MyDiscreteState(a=7.0, b=4.776488676855167)
MyContinuousState(x=0.009156362264504603, z=0.016847706566688464, theta=8.504474637647576)
MyOutput(x=0.009156362264504603, a=7.0)


## Supporting Scalar Type Conversion (double, AutoDiff, and Symbolic)

In order to support AutoDiff and Symbolic types throughout the systems framework, LeafSystems can be written to support "scalar type conversion". In Python, adding this support requires a small amount of boilerplate.  Here is a simple example:

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">state &rarr;</td></tr><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">command &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>RunningCost</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; cost</td></tr></table></td></tr></table>

In [10]:
from pydrake.systems.framework import LeafSystem_
from pydrake.systems.scalar_conversion import TemplateSystem
from pydrake.autodiffutils import AutoDiffXd
from pydrake.symbolic import Expression

@TemplateSystem.define("RunningCost_")
def RunningCost_(T):

    class Impl(LeafSystem_[T]):

        def _construct(self, converter=None, Q=np.eye(2)):
            super().__init__(converter)
            self._Q = Q
            self._state_port = self.DeclareVectorInputPort("state", 2)
            self._command_port = self.DeclareVectorInputPort("command", 1)
            self.DeclareVectorOutputPort("cost", 1, self.CostOutput)

        def _construct_copy(self, other, converter=None):
            # Any member fields (e.g. configuration values) need to be
            # transferred here from `other` to `self`.
            Impl._construct(self, converter=converter, Q=other._Q)

        def CostOutput(self, context, output):
            x = self._state_port.Eval(context)
            u = self._command_port.Eval(context)[0]
            output[0] = x.dot(self._Q.dot(x)) + u**2

    return Impl

RunningCost = RunningCost_[None]  # Default instantiation.

The important steps are:
- Add the `@TemplateSystem` decorator.
- Derive from `LeafSystem_[T]` instead of simply `LeafSystem`.
- Implement the `_construct` method *instead* of the typical `__init__` method.
- Implement the `_construct_copy` method, which needs to populate the same member fields as `_construct` (as we did with `self.Q` in this example).
- Add the default instantiation, so that you can continue to refer to the system as, e.g., `RunningCost` in addition to using `RunningCost_[float]`.

For further details, you can find the related documentation for scalar conversion in C++ [here](https://drake.mit.edu/doxygen_cxx/group__system__scalar__conversion.html) and the documentation for the [`@TemplateSystem`](https://drake.mit.edu/pydrake/pydrake.systems.scalar_conversion.html#pydrake.systems.scalar_conversion.TemplateSystem) decorator.

In [11]:
# Having done this, we can still use the system in the original way:
system = RunningCost(Q=np.diag([10, 1]))
context = system.CreateDefaultContext()
system.get_input_port(0).FixValue(context, [1, 2])
system.get_input_port(1).FixValue(context, [3])
print(system.get_output_port().Eval(context))

# But we can also now use an autodiff or symbolic versions of the system,
# either by declaring them directly:
system_ad = RunningCost_[AutoDiffXd]()
system_symbolic = RunningCost_[Expression]()

# or by scalar conversion:
system_ad = system.ToAutoDiffXd()
system_symbolic = system.ToSymbolic()

# We can also convert the time, state, parameters, and (if needed) input ports
# from the original system:
context_symbolic = system_symbolic.CreateDefaultContext()
context_symbolic.SetTimeStateAndParametersFrom(context)
system_symbolic.FixInputPortsFrom(system, context, context_symbolic)
print(system_symbolic.get_output_port().Eval(context_symbolic))

[23.]
[<Expression "23">]


## Further reading

[Caching in the systems framework](https://drake.mit.edu/doxygen_cxx/group__cache__design__notes.html)

[Declaring events](https://drake.mit.edu/doxygen_cxx/group__events__description.html)

[Stochastic systems](https://drake.mit.edu/doxygen_cxx/group__stochastic__systems.html)

Declaring [system constraints](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system_constraint.html): [equality](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html#a4948ad0241c67045b3c794874b2986a0) and [inequality](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html#a3bac306621c3f0839324649151c22af2).

[Writing continuous dynamics in implicit form](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system.html#a2bb4c1e3572a8009863b5a342fcb5c49)